# Customized Standard RAG Pipeline in LlamaIndex - with Qdrant Vector DB
* Notebook by Adam Lang
* Date: 3/22/2024
* In this notebook we will go through a customized 'standard' RAG pipeline using LlamaIndex and the Qdrant Vector DB.
* About the 'dataset' we will use for this:
  - I am going to query the privacy policy from a digital health application that has an app on the iPhone app store. The app is called the iCare HOME2 which is used for remote monitoring of glaucoma. In particular I am curious about the contents of the privacy policy and how the users data is utilized and shared. I will build a RAG application to query this.

# Steps for Customized RAG Pipeline
1. Configure different LLM
2. Use different embedding model
3. Configure different Vector Store - **Using Qdrant Vector Database**
4. Customize with different indices
5. Synthesize response for a user query

In [20]:
!pip install llama-index qdrant_client llama-index-vector-stores-qdrant

### Setup openai acess

In [5]:
import os
os.environ['OPENAI_API_KEY'] = 'your_key'

### Load dataset

In [6]:
from pathlib import Path
from llama_index.core import download_loader

# pdf reader
PDFReader = download_loader("PDFReader")

# create loader
loader = PDFReader()
documents = loader.load_data(file=Path('/content/drive/MyDrive/Colab Notebooks/Building Production Ready RAG Systems using LlamaIndex/icare_PATIENT_glaucoma_app_privacy_policy.pdf'))

<ipython-input-6-4affb446dc8f>:5: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PDFReader = download_loader("PDFReader")


In [7]:
# len document
len(documents)

13

In [10]:
# print a document
print(documents[2].text)

### Configure OpenAI LLM

In [12]:
!pip install llama-index-llms-openai

In [14]:
from llama_index.llms.openai import OpenAI
llm = OpenAI(model="gpt-3.5-turbo", temparture=0.1)


### Load BGE embeddings from HuggingFace

In [15]:
!pip install llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.6 MB/s eta 0:00:00


In [16]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [17]:
# instatiate embedding model - use BGE embeddings small model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

### Create Service Context by providing LLM and Embedding model

In [19]:
from llama_index.core import ServiceContext
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model
)

<ipython-input-19-0b686f76eddf>:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


### Configure Qdrant VectorDB

In [21]:
import qdrant_client
from llama_index.vector_stores.qdrant import QdrantVectorStore

# initialize client, setting path to save data
client = qdrant_client.QdrantClient(path="./qdrant_db")

#create collection
vector_store = QdrantVectorStore(client=client, collection_name="rag_customization")

### Create Store Context by assigning vector store created

In [23]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)

# 1. VectorStore Index
* Define the vector store index by passing storage context and service context

In [54]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents[:200],
                                        storage_context=storage_context,
                                        service_context=service_context,
                                        show_progress=True)

Parsing nodes:   0%|          | 0/13 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/13 [00:00<?, ?it/s]

### Build the query engine for the index

In [55]:
query_engine = index.as_query_engine(similarity_top_k=5)

In [56]:
response=query_engine.query(
    "Is the users data shared with a 3rd party?"
)

In [57]:
# print response
print(response)

The user's data is not shared with a 3rd party.


# 2. Keyword Table

In [97]:
from llama_index.core import KeywordTableIndex

keyword_table_index = KeywordTableIndex.from_documents(
    documents[:600],
    service_context=service_context,
    show_progress=True

)

Parsing nodes:   0%|          | 0/13 [00:00<?, ?it/s]

Extracting keywords from nodes:   0%|          | 0/13 [00:00<?, ?it/s]

In [98]:
# build retriever
keyword_table_retriever = keyword_table_index.as_retriever()

In [99]:
# build query engine
query_engine = keyword_table_index.as_query_engine()

In [111]:
# response query
response = query_engine.query(
    """What is the name of the device?"""
)

In [112]:
print(response)

The name of the device is "icare_PATIENT_glaucoma_app_privacy_policy.pdf".
